In [171]:
import os
import csv
import json
import requests
import pandas as pd
import numpy as np
import configparser

In [2]:
config = configparser.ConfigParser()
config.read(".env")
APIKEY = config["RAPIDAPI"]["KEY"]

## Outline of Data Extraction Process

 ---
 
 Using the RapidApi, the data will be collected in the following order.
 
 * The symbols of market movers will be gotten using the endpoint "market/v2/get-movers" 
 * The market movers will be added as query to the endpoint "stock/v3/get-historical-data" to collect historical data of the tickers
 * The symbols of the market movers will also be used as query to get their summary which includes the financial earnings using the endpoint stock/v2/get-summary
 * Currency pairs selected for the project are used as query in the endpoint "market/get-spark" to collect data according to interval and range. 
 * The collected data will initially be saved in pandas dataframe.  

#### Extract 10 stock losers, gainers and most-active stocks in GB region, get their symbols 

In [3]:
url = "https://yh-finance.p.rapidapi.com/market/v2/get-movers"

querystring = {"region":"GB","lang":"en-GB","count":"10","start":"0"}

headers = {
	"X-RapidAPI-Key": APIKEY,
	"X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring).json()

In [4]:
# Get the gainers from quotes
gainer_quotes = response["finance"]["result"][0]["quotes"]
gainers = []

for quote in gainer_quotes:
    gainers.append(quote["symbol"]) 

In [5]:
# Get the losers
losers_quotes = response["finance"]["result"][1]["quotes"]
losers = []

for quote in losers_quotes:
    losers.append(quote["symbol"]) 

In [6]:
# Get the most actives
mostactives_quotes = response["finance"]["result"][2]["quotes"]
mostactives = []

for quote in mostactives_quotes:
    mostactives.append(quote["symbol"]) 

In [14]:
# Combine all into one DF
movers_df = pd.DataFrame(list(zip(gainers, losers, mostactives)), columns=['gainers', 'losers', 'mostactives'])

movers_df

,gainers,losers,mostactives
0,0BDR.IL,0JI3.L,7DIG.L
1,0MC5.IL,0E2B.IL,ICON.L
2,0RCP.IL,0AAT.IL,KOD.L
3,TUIB.L,MH65.L,SYME.L
4,0RCS.IL,0AAS.IL,0VRF.L
5,0RCR.IL,0V6Y.L,0MRI.IL
6,0XC6.IL,0OI0.L,PREM.L
7,0E4B.IL,0A5O.IL,0RQY.L
8,0I21.L,0MN3.IL,BOIL.L
9,0DP0.IL,0XWG.IL,GST.L


In [15]:
# Save movers in csv
movers_df.to_csv("data/movers.csv", index=False)

#### Extract historical data with the symbols 

In [3]:
#import movers csv
movers_df = pd.read_csv("data/movers.csv")

In [18]:
def history_data(path, movers, key):
    """
    
    :return: None
    """
    data = []
    url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"
    for symbol in movers_df[movers]: 
        querystring = {"symbol": symbol,"region":"GB"}

        headers = {
            "X-RapidAPI-Key": key,
            "X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring).json()
        
        names = ['date', 'open', 'high', 'low', 'close', 'volume', 'adjclose', 'symbol', 'movers']
        for stock in response["prices"]:
            if len(stock) == 7:
                history_data = {'date': stock['date'], 
                                'open': stock['open'], 
                                'high': stock['high'], 
                                'low': stock['low'],
                                'close': stock['close'], 'volume': stock['volume'], 
                                'adjclose': stock['adjclose'], 'symbol': symbol,
                                'movers': movers}
                
                data.append(history_data)
        with open(path, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=names)
            writer.writeheader()
            writer.writerows(data)

In [19]:
# Collect historical data of each stock
history_data("data/gainers_history.csv", "gainers", APIKEY)
history_data("data/losers_history.csv", "losers", APIKEY)
history_data("data/actives_history.csv", "mostactives", APIKEY)

In [240]:
#combine history stocks data into one dataframe
gainers_history = pd.read_csv("data/gainers_history.csv")
losers_history = pd.read_csv("data/losers_history.csv")
actives_history = pd.read_csv("data/actives_history.csv")

stocks_history = pd.concat([gainers_history,losers_history, actives_history], ignore_index=True)
stocks_history.to_csv("data/stocks_history.csv", index=False)

#### Extract Summary and Financial information of each stock

In [266]:
def quote_summary_earnings(path, movers):
    """
    :return: None
    """
    data = []
    cols = ["symbol", "summary", "quoteType", "beta", "dividendRate", "marketCap","dividendYield","exDividendDate","dayHigh",
           "dayLow","ask", "previousClose", "marketOpen", "bid", "askSize", "bidSize", "volume", "fiftyTwoWeekHigh", "fiftyTwoWeekLow",
           "earnings"]
    for symbol in movers_df[movers]: 
        url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"

        querystring = {"symbol":symbol,"region":"US"}

        headers = {
            "X-RapidAPI-Key": APIKEY,
            "X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring).json()
        summaryDetail = response.get("summaryDetail", {})
        try:
            profile_earnings = {
               "symbol": symbol,
               "summary": response.get("summaryProfile", {}),
               "quoteType": response.get("quoteType", {}).get("quoteType", ""),
               "beta": summaryDetail.get("beta", {}).get("raw", 0),
               "dividendRate": summaryDetail.get("dividendRate",{}).get("raw", 0),
               "marketCap": summaryDetail.get("marketCap", {}).get("raw", 0),
               "dividendYield": summaryDetail.get("dividendYield", {}).get("raw", 0),
               "exDividendDate": summaryDetail.get("exDividendDate", {}).get("raw", 0),
               "dayHigh": summaryDetail.get("dayHigh", {}).get("raw", 0),
               "dayLow": summaryDetail.get("dayLow", {}).get("raw", 0),
               "ask": summaryDetail.get("ask", {}).get("raw", 0),
               "previousClose": summaryDetail.get("previousClose", {}).get("raw", 0),
               "marketOpen": summaryDetail.get("open", {}).get("raw", 0),
               "bid": summaryDetail.get("bid", {}).get("raw", 0),
               "askSize": summaryDetail.get("askSize", {}).get("raw", 0),
               "bidSize": summaryDetail.get("bidSize", {}).get("raw", 0),
               "volume": summaryDetail.get("volume", {}).get("raw", 0),
               "fiftyTwoWeekHigh": summaryDetail.get("fiftyTwoWeekHigh", {}).get("raw", 0),
               "fiftyTwoWeekLow": summaryDetail.get("fiftyTwoWeekLow", {}).get("raw", 0),
               "earnings": response.get("earnings", {}).get("financialsChart", {}).get("yearly", {})
            }
            data.append(profile_earnings) 
        except Exception as E:
            print(f'There is an error {E} in the data')
            
        with open(path, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=cols)
            writer.writeheader()
            writer.writerows(data)

In [267]:
quote_summary_earnings("data/gainers_profile_earnings.csv", "gainers")
quote_summary_earnings("data/losers_profile_earnings.csv", "losers")
quote_summary_earnings("data/actives_profile_earnings.csv", "mostactives")

In [268]:
# combine stocks profile earnings data into stock_profile_earnings
gainers_profile_earnings = pd.read_csv("data/gainers_profile_earnings.csv", encoding='latin1')
losers_profile_earnings = pd.read_csv("data/losers_profile_earnings.csv",encoding='latin1' )
actives_profile_earnings = pd.read_csv("data/actives_profile_earnings.csv",encoding='latin1')

stock_profile_earnings = pd.concat([gainers_profile_earnings, losers_profile_earnings, actives_profile_earnings], ignore_index=True)
stock_profile_earnings.to_csv("data/stock_profile_earnings.csv", index=False)

#### Extract Historical data of 10 currency pairs

 
The currency data will be collected as follows
 * Extract records using endpoint "market/get-spark"
 * Get 1min - 5day, 5min - 1mo, 1day - 6mo and 1wk - 1yr interval range of each pair
 * Extract according to interval. Combine later
 * The currencies symbol include: 
   GBPAUD=X,GBPUSD=X,GBPEUR=X,GBPJPY=X,GBPCAD=X,EURJPY=X,EURGBP,EURUSD=X,AUDUSD=X,NZDUSD=X 

In [222]:
def get_currencies(path, interval, rang, currencies):
    """
        
   """ 
    url = "https://yh-finance.p.rapidapi.com/market/get-spark"

    querystring = {"symbols": (",").join(currencies), "interval":interval, "range": rang}

    headers = {
    	"X-RapidAPI-Key": APIKEY,
    	"X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring).json()
    data = []
    cols = ["symbol", "timestamp", "close", "endTime", "startTime", "chartPrevClose", "interval", "range"]
    for pair in response:
        for timeDur in range(len(response[pair]["timestamp"])):
            currency_data = {
                "symbol": pair,
                "timestamp": response[pair]["timestamp"][timeDur],
                "close": response[pair]["close"][timeDur],
                "endTime": response[pair]["end"],
                "startTime": response[pair]["start"],
                "chartPrevClose": response[pair]["chartPreviousClose"],
                "interval": interval,
                "range": rang
            }
            
            data.append(currency_data)
    with open(path, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=cols)
        writer.writeheader()
        writer.writerows(data)     

In [223]:
currencies = ["GBPAUD=X","GBPUSD=X","GBPEUR=X","GBPJPY=X","GBPCAD=X","EURJPY=X","EURGBP=X","EURUSD=X","AUDUSD=X","NZDUSD=X"]

In [224]:
get_currencies("data/interRang1m5d.csv", "1m", "5d", currencies)
get_currencies("data/interRang5m1mo.csv", "5m", "1mo", currencies)
get_currencies("data/interRang1d6mo.csv", "1d", "6mo", currencies)
get_currencies("data/interRang1wk1y.csv", "1wk", "1y", currencies)

In [274]:
#historical_currencies
interRang1m5d = pd.read_csv("data/interRang1m5d.csv")
interRang5m1mo = pd.read_csv("data/interRang5m1mo.csv")
interRang1d6mo = pd.read_csv("data/interRang1d6mo.csv")
interRang1wk1y = pd.read_csv("data/interRang1wk1y.csv")

history_currencies = pd.concat([interRang1m5d,interRang5m1mo,interRang1d6mo,interRang1wk1y], ignore_index=True)
history_currencies.to_csv("data/history_currencies.csv", index=False)

## Data Transformation
---

The profile and earning data will be transformed to extract data from any imbedded dictonary, the following will be done and achieved.

* Check dictionary columns and collect relevant data. If dictionary doesn't contain a value, use a substitute.
* Decide which records are needed, create new dataframe if columns are many
  

## Outline of Data Cleaning Process

---

  The collected data will here be cleaned and processed.
  The historical stocks, currencies and stocks profile and earnings data will be processed first by the following steps
  
  * Combine the related DataFrames
  * Check for missing or Null values in the columns
  * Remove or substitute for missing and null values
  * Check statistics of data
  * Save cleaned data in DF
  
  

## Data Modelling, Loading and Storage